In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import itertools
data_path = Path('~/Documents/gpl/eop/data')

def get_row_from_metadata(metadata, covariate_name):
    """
    Extracts a specific row from the metadata DataFrame based on the covariate name.

    :param metadata: DataFrame containing metadata.
    :param covariate_name: Name of the covariate to extract.
    :return: Row corresponding to the specified covariate name.
    """
    return metadata.loc[metadata['variable_name'] == covariate_name].squeeze()

def all_rows_from_metadata_containing(metadata, substring):
    """
    Extracts all rows from the metadata DataFrame that contain a specific substring in the variable name.

    :param metadata: DataFrame containing metadata.
    :param substring: Substring to search for in the variable names.
    :return: DataFrame containing all rows with variable names that contain the substring.
    """
    return metadata[metadata['variable_name'].str.contains(substring, na=False)].reset_index(drop=True)

def all_column_names_containing(df, substring):
    """
    Extracts all column names from the DataFrame that contain a specific substring.

    :param df: DataFrame to search for column names.
    :param substring: Substring to search for in the column names.
    :return: List of column names containing the specified substring.
    """
    return [col for col in df.columns if substring in col]


def find_equivalent_columns(data, numeric_tolerance=1e-6, categorical_threshold=0.99):
    """
    Find pairs of columns in a DataFrame that are informationally equivalent.
    
    Parameters:
    -----------
    data : pandas DataFrame
        The DataFrame to analyze
    numeric_tolerance : float, default 1e-6
        Tolerance for considering numeric columns equal or proportional
    categorical_threshold : float, default 0.99
        Threshold for considering categorical columns equivalent (percentage match)
    
    Returns:
    --------
    list of tuples
        Each tuple contains (col1, col2, relationship_type)
        where relationship_type is one of: 'identical', 'proportional', 'categorical_equivalent'
    """
    equivalent_pairs = []
    columns = data.columns
    
    # Get column types
    numeric_cols = data.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()

    # Identify constant columns
    constant_cols = []
    for col in columns:
        unique_values = data[col].dropna().unique()
        if len(unique_values) <= 1:
            constant_cols.append(col)




    # Print constant columns if verbose
    if len(constant_cols) > 0:
        print("Constant columns:")
        for col, value in constant_cols:
            print(f"  '{col}': {value}")
        print()
    

    # Remove constant columns from numeric and categorical lists
    numeric_cols = [col for col in numeric_cols if col not in constant_cols]
    categorical_cols = [col for col in categorical_cols if col not in constant_cols]
    
    # Check numeric columns for equality or proportionality
    for col1, col2 in itertools.combinations(numeric_cols, 2):

        # Check for identical values first
        if data[col1].equals(data[col2]):
            equivalent_pairs.append((col1, col2, 'identical'))
            continue
            
        # Check for identical values where neither is zero
        valid_mask = ~data[col1].isna() & ~data[col2].isna()
        if np.allclose(data.loc[valid_mask, col1], data.loc[valid_mask, col2], 
                      rtol=numeric_tolerance, atol=numeric_tolerance):
            equivalent_pairs.append((col1, col2, 'nearly_identical'))
            continue
        
        # For rows with zeros, check if the columns are exactly equal
        zero_mask = (data[col1] == 0) | (data[col2] == 0)
        non_zero_mask = ~zero_mask & valid_mask
        
        # Check if the columns have the same values where zeros are present
        if zero_mask.any():
            zero_equality = (data.loc[zero_mask & valid_mask, col1] == 
                             data.loc[zero_mask & valid_mask, col2]).all()
        else:
            zero_equality = True
            
        # Check for proportional relationship in non-zero values
        if non_zero_mask.sum() > 10:  # Require at least some non-zero values
            ratios = data.loc[non_zero_mask, col2] / data.loc[non_zero_mask, col1]
            ratio_std = ratios.std()
            
            # If standard deviation of ratios is very small, columns are proportional
            if ratio_std < numeric_tolerance and zero_equality:
                ratio = ratios.mean()
                equivalent_pairs.append((col1, col2, f'proportional (factor: {ratio:.4f})'))
    
    # Create a list of all columns to check for categorical equivalence
    # This includes both explicit categorical columns and numeric columns
    all_potential_categorical_cols = categorical_cols + numeric_cols
    
    # Check all columns for equivalent categorical mappings
    for col1, col2 in itertools.combinations(all_potential_categorical_cols, 2):
        # Skip if identical columns or already identified as identical or proportional
        if col1 == col2 or any((col1, col2, rel) in equivalent_pairs for rel in 
                               ['identical', 'nearly_identical', 'proportional']):
            continue
            
        # Get unique values for both columns
        unique_vals1 = data[col1].dropna().unique()
        unique_vals2 = data[col2].dropna().unique()
        
        # Skip if columns have different number of unique values
        if len(unique_vals1) != len(unique_vals2):
            continue
            
        # Skip if too many unique values (likely not categorical)
        if len(unique_vals1) > 100:  # Arbitrary threshold, adjust as needed
            continue
            
        # Create a mapping table between values in both columns
        mapping_df = data[[col1, col2]].dropna().drop_duplicates()
        
        # Check if mapping is one-to-one (each value in col1 maps to exactly one value in col2)
        is_one_to_one = True
        
        # Check col1 -> col2 mapping
        for val in unique_vals1:
            corresponding_vals = data.loc[data[col1] == val, col2].dropna().unique()
            if len(corresponding_vals) != 1:
                is_one_to_one = False
                break
                
        # Check col2 -> col1 mapping
        if is_one_to_one:
            for val in unique_vals2:
                corresponding_vals = data.loc[data[col2] == val, col1].dropna().unique()
                if len(corresponding_vals) != 1:
                    is_one_to_one = False
                    break
        
        if is_one_to_one:
            # If we create a new column using the mapping, it should match the original
            val_mapping = dict(zip(mapping_df[col1], mapping_df[col2]))
            
            # Apply mapping and handle NaN values
            mapped_values = data[col1].map(val_mapping)
            
            # Count matches (ignoring NaN values)
            valid_mask = ~data[col1].isna() & ~data[col2].isna()
            if valid_mask.sum() > 0:
                match_percentage = (mapped_values == data[col2])[valid_mask].mean()
                
                if match_percentage >= categorical_threshold:
                    # Determine if both are numeric or mixed types
                    if col1 in numeric_cols and col2 in numeric_cols:
                        relationship = 'numeric_categorical_equivalent'
                    else:
                        relationship = 'categorical_equivalent'
                    equivalent_pairs.append((col1, col2, relationship))
    
    return equivalent_pairs

Done here
- Ensure that missingness-indicator columns exist.
    - You probably can't conclusively check that all are included, because the data you get will not necessarily reveal which columns had missingness, but check that there are some missingness columns, and none for categorical data.
- Ensure there are no NaNs in the data.
- Ensure column names:
    - In data: "hhid" (if household ID is included), "consumption_per_capita_per_day", "hh_wgt".
    - Consumption: Check mean and std for sanity. In a poor country, the mean should be low-mid single digits: e.g., in Uganda, the mean is $3.80/day.
- Check for columns that indicate units:
    - If they are present, the corresponding numeric field should be standardized, e.g., all area units adjusted to square meters.
- Check that metadata and the dataset itself match:
    - Every column in data is described in metadata and vice versa. It's also OK if `hhid` is not in the data at all.
- In metadata:
    - "variable_name".
    - "data_type", with permitted values "numeric" and "categorical".
    - "geographic_indicator".
- Scan datatypes:
    - In particular, make sure nothing is numeric which should be categorical.
    - Ensure categorical-type columns have the appropriate type even if the categories are encoded as integers (if a column is binary, with no missing values, it can be numeric or categorical).
    - IDs of all kinds are strings even if they appear numeric.

- Check for duplication

In [140]:
if False: # Albania
    data = pd.read_parquet(
        data_path / 'albania' / 'cleaned' / 'albania_all.parquet'
    )
    summary = pd.read_parquet(
        data_path / 'albania' / 'cleaned' / 'summary.parquet'
    )
if True: # Uganda
    data = pd.read_parquet(
        data_path / 'uganda' / 'cleaned' / 'uganda_full.parquet'
    )
    summary = pd.read_parquet(
        data_path / 'uganda' / 'cleaned' / 'summary.parquet'
    )

In [141]:
# nullity
display(data.isna().mean().sort_values().head(1))

hhid    0.0
dtype: float64

In [142]:
# Missingness columns (assumes _missing suffix)
missingness_columns = [c for c in data.columns if 'missing' in c]
with_missingness = [c[:-8] for c in missingness_columns]
for c in with_missingness:
    if not (c in summary.variable_name.values):
        print(f"Missingness column {c} not in summary")
relevant_summary = summary[summary.variable_name.isin(with_missingness)]
print(relevant_summary.data_type.value_counts())
display(relevant_summary[relevant_summary.data_type == 'categorical'])

data_type
numeric        4
categorical    1
Name: count, dtype: int64


,variable_name,module_name,variable_description,data_type,geographic_indicator
32,highest_education_household,custom aggregate variables,Highest level of education in household,categorical,False


In [143]:
# Key columns; consumption
assert 'hhid' in data.columns
assert 'consumption_per_capita_per_day' in data.columns
print(data.consumption_per_capita_per_day.mean())
print(data.consumption_per_capita_per_day.std())

3.801285894547133
3.91231662305722


In [144]:
# Suspicious data
display(
    summary[
        summary.variable_name.str.contains('unit')
        | summary.variable_description.str.contains('unit')
    ]
)
display(
    summary[
        summary.variable_name.str.contains('consumption')
        | summary.variable_description.str.contains('consumption')
    ]
)

,variable_name,module_name,variable_description,data_type,geographic_indicator


,variable_name,module_name,variable_description,data_type,geographic_indicator
90,consumption_per_capita_per_day,custom aggregate variables,Daily consumption of household per capita in 2...,numeric,False


In [145]:
# check that metadata and data match
data_columns = set(data.columns)

summary_variable_names = set(summary['variable_name'])
missing_in_data = summary_variable_names - data_columns
missing_in_summary = data_columns - summary_variable_names

print("Variables in summary but not in data:", missing_in_data)
print("Columns in data but not in summary:", missing_in_summary)

Variables in summary but not in data: set()
Columns in data but not in summary: {'hhid'}


In [146]:
# Check that "summary" fits the required format
required_columns = {"variable_name", "data_type", "geographic_indicator"}
summary_columns = set(summary.columns)

missing_columns = required_columns - summary_columns
if missing_columns:
    print(f"Missing required columns in summary: {missing_columns}")

# Ensure "data_type" has only permitted values
permitted_data_types = {"numeric", "categorical"}
for _, row in summary.iterrows():
    if row["data_type"] not in permitted_data_types:
        print(
            f"Invalid data_type '{row['data_type']}' for variable '{row['variable_name']}'. "
            f"Description: {row['variable_description']}"
        )

# Ensure "geographic_indicator" is boolean or 0-1
for _, row in summary.iterrows():
    if row["geographic_indicator"] not in [0, 1, True, False, None]:
        print(
            f"Invalid geographic_indicator '{row['geographic_indicator']}' for variable '{row['variable_name']}'. "
            f"Description: {row['variable_description']}"
        )

In [147]:
# Check that numeric columns in summary are actually numeric in data
numeric_columns = summary[summary["data_type"] == "numeric"]["variable_name"]
for col in numeric_columns:
    if col in data.columns and not pd.api.types.is_numeric_dtype(data[col]):
        description = summary.loc[summary["variable_name"] == col, "variable_description"].values[0]
        print(f"numeric in summary, non-numeric in data: '{col}'; {description}")

# Check that categorical columns in summary are actually categorical in data
categorical_columns = summary[summary["data_type"] == "categorical"]["variable_name"]
for col in categorical_columns:
    if col in data.columns and not pd.api.types.is_categorical_dtype(data[col]):
        description = summary.loc[summary["variable_name"] == col, "variable_description"].values[0]
        print(f"categorical in summary, numeric in data: '{col}'; {description}")

categorical in summary, numeric in data: 'region'; Region of Residence in 2019/20
categorical in summary, numeric in data: 'subreg'; Sub region in 2019/20
categorical in summary, numeric in data: 'district'; District Code
categorical in summary, numeric in data: 's1aq02a'; County/Municipality
categorical in summary, numeric in data: 's1aq03a'; Sub-County/Division/ Town Council
categorical in summary, numeric in data: 's1aq04a'; Parish/Ward
categorical in summary, numeric in data: 'urban'; Rural/Urban
categorical in summary, numeric in data: 'highest_education_household'; Highest level of education in household
categorical in summary, numeric in data: 'h9q01'; What type of dwelling is it?
categorical in summary, numeric in data: 'h9q04'; What is the major construction material of the roof?
categorical in summary, numeric in data: 'h9q05'; What is the major construction material of the external wall?
categorical in summary, numeric in data: 'h9q06'; What is the major material of the floo

/var/folders/l3/jdqjbmvx4c39mkfwcnm559y00000gn/T/ipykernel_6730/1155484756.py:11: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if col in data.columns and not pd.api.types.is_categorical_dtype(data[col]):


In [148]:
# Print variables whose name contains "id" or "code" and are listed as numeric in the summary
filtered_variables = summary[
    (summary["variable_name"].str.contains("id|code", case=False, na=False)) &
    (summary["data_type"] == "numeric")
]

# Print the name and description of the filtered variables
for _, row in filtered_variables.iterrows():
    print(f"Name: {row['variable_name']}, Description: {row['variable_description']}")

Name: Crop Residue_cooking, Description: Household uses Crop Residue for cooking
Name: Crop Residue_lighting, Description: Household uses Crop Residue for lighting
Name: Crop Residue_heating, Description: Household uses Crop Residue for heating


In [161]:
# Check for duplicate information
find_equivalent_columns(data)

Constant columns: ['total_head_household_members', 'transport_equipment_owned', 'washing_machine_dryer_owned', 'Torches_cooking', 'Torches_heating', 'lpg_lighting', 'lpg_heating', 'Dung_lighting', 'Dung_heating']


[]

In [154]:
data[['lpg_lighting', 'Dung_lighting']].value_counts()

lpg_lighting  Dung_lighting
0             0                3074
Name: count, dtype: int64